In [37]:
# !pip install --upgrade "kfp==2.0.1"  --quiet
# # 重新启动 kernel，再 import
# %pip install --quiet --upgrade "kfp==2.0.1"


!pip uninstall -y kfp
!pip install 'kfp==2.9.0'
# !pip uninstall -y kfp
# !pip install 'kfp==1.8.22'

import kfp, sys

print("KFP version →", kfp.__version__)

Found existing installation: kfp 2.9.0
Uninstalling kfp-2.9.0:
  Successfully uninstalled kfp-2.9.0
  Using cached kfp-2.9.0-py3-none-any.whl
KFP version → 2.9.0


In [44]:
# section 1
from kfp import compiler
from pipeline.pipeline import drift_demo_pipeline

compiler.Compiler().compile(
    pipeline_func=drift_demo_pipeline,
    package_path='drift_demo_v4.yaml'
)
print("✅ drift_demo_v4.yaml 已生成")



✅ drift_demo_v4.yaml 已生成


In [45]:
# # section 3
# token = os.environ["MY_TOKEN"]
# client = kfp.Client(
#     host="http://45.149.207.13:38183/pipeline",
#     existing_token=token,
#     namespace="kubeflow"
# )

# # 验证能否列出实验
# print(client.list_experiments().experiments)

import time
import kfp
from kfp_server_api.exceptions import ApiException

# 不指定 host，使用默认 in-cluster 服务
client = kfp.Client()


# print(client.list_experiments().experiments)
# 3. 列出已有实验
experiments = client.list_experiments().experiments
print("现有实验：", [e.display_name for e in experiments])

现有实验： ['Latency-Pipeline-Experiment', 'Default', 'auto_digits_pipeline_half_simplified', 'auto_digits_pipeline_demo2', 'katib-test0', 'katib-flux-demo', 'twin-stream', 'trial-exp', 'drift-detect-demo-exp', 'drift-detect-demo-v2-exp']


In [47]:
import time
import os
from kfp import Client
from kfp_server_api.exceptions import ApiException

# 1. 初始化客户端（in-cluster 模式会自动挂载 ServiceAccount Token）
client = kfp.Client()

# 2. 幂等上传或复用已有的 Pipeline ID
pipeline_name = "drift-detect-demo-v4"
yaml_path = "drift_demo_v4.yaml"

try:
    # 上传新的 Pipeline（若不存在）
    pipeline_id = client.upload_pipeline(
        pipeline_package_path=yaml_path,
        pipeline_name=pipeline_name
    ).pipeline_id
    print(f"✅ 已上传新 Pipeline，ID = {pipeline_id}")
except ApiException as e:
    if e.status == 409:
        # 如果已经存在同名 Pipeline，复用其 ID
        pipeline_id = client.get_pipeline_id(pipeline_name)
        print(f"⚠️ Pipeline 重名，复用已有 ID = {pipeline_id}")
    else:
        # 其他异常继续抛出
        raise

# 3. 提交一个新的 Run
run_name = f"{pipeline_name}-run-{int(time.time())}"
run_result = client.create_run_from_pipeline_package(
    pipeline_file=yaml_path,
    arguments={},               # 若有参数则填入字典
    run_name=run_name,
    experiment_name=pipeline_name + "-exp",  # 可根据需求指定 Experiment 名称
    enable_caching=False  # ← 关闭整个 Run 的缓存
)

print("▶️ Run 已提交")
print("Run ID  =", run_result.run_id)
print("Run URL =", f"{client._get_url_prefix()}/#/runs/details/{run_result.run_id}")


⚠️ Pipeline 重名，复用已有 ID = cd24c421-f90e-4070-adc1-aca19e3e5720


▶️ Run 已提交
Run ID  = 5349867b-eea9-4cf6-ba06-b6811b5417ca
Run URL = /pipeline/#/runs/details/5349867b-eea9-4cf6-ba06-b6811b5417ca


In [ ]:
import time
from kfp import Client

client = kfp.Client()

# 1. 上传或复用已有 pipeline_id（略过重复代码）
pipeline_id = "你前面拿到的 pipeline_id"

# 2. 提交 Run
run_result = client.create_run_from_pipeline_package(
    pipeline_file="drift_demo.yaml",
    arguments={},  
    run_name=f"drift-demo-{int(time.time())}",
    experiment_name="drift-detect-demo-exp"
)

print("Run ID:", run_result.run_id)
print("Run URL:", f"{client._get_url_prefix()}/#/runs/details/{run_result.run_id}")


In [26]:
# section 4
import kfp
# client = kfp.Client(
#     host="http://45.149.207.13:38183/pipeline",
#     existing_token=MY_TOKEN,
#     namespace="kubeflow"        # 默认都填 kubeflow
# )



# upload pipeline (IDempotent)
pipeline_id = client.upload_pipeline(
    pipeline_package_path="drift_demo.yaml",
    pipeline_name="drift-detect-demo"
).id

# kick off a run
run = client.create_run_from_pipeline_id(
    pipeline_id=pipeline_id,
    run_name="drift-demo-run"
)
print("Run URL:", f"{client._get_url_prefix()}/#/runs/details/{run.run_id}")



AttributeError: 'V2beta1Pipeline' object has no attribute 'id'

In [ ]:
# section2
client = kfp.Client(host="http://45.149.207.13:38183/pipeline")

# 上传（第一次）或获取已上传 ID
pipeline_id = client.upload_pipeline(
    pipeline_package_path="drift_demo.yaml",
    pipeline_name="drift-detect-demo"
).id

# 启动一次运行
run = client.create_run_from_pipeline_id(
    pipeline_id=pipeline_id,
    run_name="drift-demo-run"
)
print("Run URL:",
      f"{client._get_url_prefix()}/#/runs/details/{run.run_id}")
